In [299]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

In [300]:
# 學生預測結果檔案

# Sample
student_submit_ner   = './data/student_prediction/sample_brand.csv'                 # 品牌預測結果 (sample)
student_submit_multi = './data/student_prediction/sample_cat.csv'                   # 品類預測結果 (sample)
zz = pd.read_csv('./data/brand_list/brand_list_standard.csv')          # 品牌表 (標準檔案)

# 第一組
#student_submit_ner   = './data/student_prediction/brand_test_team1.csv'     # 品牌預測結果 (第一組)
#student_submit_multi = './data/student_prediction/cat_test_team1.csv'       # 品類預測結果 (第一組)
#zz = pd.read_csv('./data/brand_list/brand_list_team1_and_team2.csv')  # 第一組 & 第二組的品牌表 (與 brand_list_standard.csv 合併)

# 第二組
#student_submit_ner   = './data/student_prediction/brand_test_team2.csv'     # 品牌預測結果 (第二組)
#student_submit_multi = './data/student_prediction/cat_test_team2.csv'       # 品類預測結果 (第二組)
#zz = pd.read_csv('./data/brand_list/brand_list_team1_and_team2.csv')  # 第一組 & 第二組的品牌表 (與 brand_list_standard.csv 合併)

# 第三組
#student_submit_ner   = './data/student_prediction/brand_test_team3.csv'     # 品牌預測結果 (第三組)
#student_submit_multi = './data/student_prediction/cat_test_team3.csv'       # 品類預測結果 (第三組)
#zz = pd.read_csv('./data/brand_list/brand_list_team3.csv')            # 第三組的品牌表 (與 brand_list_standard.csv 合併)

# 第四組
#student_submit_ner   = './data/student_prediction/brand_test_team4.csv'     # 品牌預測結果 (第四組)
#student_submit_multi = './data/student_prediction/cat_test_team4.csv'       # 品類預測結果 (第四組)
#zz = pd.read_csv('./data/brand_list/brand_list_team4.csv')            # 第四組的品牌表 (與 brand_list_standard.csv 合併)

## 品牌計分

* aka mapping

In [301]:
# 載入品牌 mapping 表 (aka表)
df_train = pd.read_csv('./data/dataset/brand_train_new.csv',usecols=['name','brand'])
df_test  = pd.read_csv('./data/dataset/brand_test_new(ans).csv',usecols=['name','brand'])
old_brand = set([x for x in df_train['brand']])

In [302]:
# 處理「brand_list_standard.csv」資料，將之轉換成 brand dictionary
entity_map = {}
for row in zz.values:
    tmp_list = list(set([x.strip() for x in row[:] if str(x)!='nan']))
    tmp_list = sorted(tmp_list,key=lambda x:len(x.strip()))
    entity_map[tmp_list[0].strip()] = tmp_list#[1:]

brand_remap = {}
for k,v in entity_map.items():
    for _v in v:
        brand_remap[_v] = k

In [303]:
aka_ans = []
for ele in df_test['brand']:
    uniq_brand = brand_remap.setdefault(ele,ele)
    aka_ans.append(entity_map.setdefault(uniq_brand,[uniq_brand]))

df_test['aka_ans'] = aka_ans
df_test['old_tag'] = df_test['aka_ans'].apply(lambda x: int(any([y in old_brand for y in x])))
df_test['true'] = df_test['brand'].apply(lambda x: brand_remap.setdefault(x,x))

* load student answer

In [304]:
df_submit_ner = pd.read_csv(student_submit_ner)

In [305]:
df_test['pred'] = df_submit_ner['brand_pred'].fillna('')
df_test['pred'] = df_test['pred'].apply(lambda x: brand_remap.setdefault(x,x))

## scoring

* 既有標籤

In [306]:
ner_score1 = f1_score(y_true=df_test.loc[df_test['old_tag']==1,'true'],
                      y_pred=df_test.loc[df_test['old_tag']==1,'pred'],
                      average="macro")

* 未知標籤

In [307]:
def jaccard(str1, str2):
    a = set(str1.lower().replace(' ',''))
    b = set(str2.lower().replace(' ',''))
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def max_jaccard(true_list,pred_str):
    out = []
    for _true,_pred in zip(true_list,pred_str):
        out.append(max([jaccard(sub_true,_pred) for sub_true in _true]))
    return np.mean(out)

In [308]:
ner_score2 = max_jaccard(true_list=df_test.loc[df_test['old_tag']==0,'aka_ans'],
                         pred_str =df_test.loc[df_test['old_tag']==0,'pred'])

# 分類計分

In [309]:
def onehot(array):
    n,p = len(array),len(label_map)
    empty_array = np.zeros((n,p),dtype=int)
    for i in range(n):
        for j in str(array[i]).split(';'):
            if label_map.get(j):
                empty_array[i,label_map[j]] = 1
    return empty_array

In [310]:
df = pd.read_csv('./data/dataset/cat_test_v2(ans).csv',usecols=['name','product'])
label_map = {v:i for i,v in enumerate(set(sum([x.split(';') for x in df['product'].unique()],[])))}

In [311]:
df_submit_multi =  pd.read_csv(student_submit_multi).fillna('')

In [312]:
multi_score1 = f1_score(y_true=onehot(df['product'].tolist()),
                        y_pred=onehot(df_submit_multi['pred'].tolist()),
                        average="macro",
                        zero_division=0)

# 總計分

In [313]:
print(f"[品類任務] Macro F1: {multi_score1:.6f}, 項目總分: {multi_score1:.6f}")
print(f"[品牌任務] 既有品牌 Macro F1: {ner_score1:.6f}, 未知品牌 Jaccard :{ner_score2:.6f}, 項目總分: {ner_score1+ner_score2*1.2:.6f}")
print('--'*40)
#print(f"[總得分]  {multi_score1+ner_score1+ner_score2*1.2:.6f}")
print(f"[總得分(加權)] {multi_score1*0.5+(ner_score1+ner_score2*1.2)*0.5:.6f}")
print("總得分計算 = (品類 Marco F1)*50% + (計有品牌 Marco F1 + 未知品牌 Jaccard * 1.2)*50%")

[品類任務] Macro F1: 0.960945, 項目總分: 0.960945
[品牌任務] 既有品牌 Macro F1: 0.117373, 未知品牌 Jaccard :0.655351, 項目總分: 0.903794
--------------------------------------------------------------------------------
[總得分(加權)] 0.932369
總得分計算 = (品類 Marco F1)*50% + (計有品牌 Marco F1 + 未知品牌 Jaccard * 1.2)*50%
